In [3]:
import pandas as pd
import numpy as np

from gym import spaces, logger

In [5]:
online_shoopers_intention = pd.read_csv('./online_shoppers_intention.csv')
# 
online_shoopers_intention.loc[:,online_shoopers_intention.select_dtypes(bool).columns] = online_shoopers_intention.select_dtypes(bool).astype(int)
# .astype()

df = online_shoopers_intention.select_dtypes(np.number)
episode_df= df.sample(frac=0.5)
# episode=episode_df.drop(columns=['Revenue']).values.tolist()
true_labels =episode_df['Revenue'].values.tolist()
        

In [6]:
lamb_da = 1908/10422
class CartPoleEnv(gym.Env):
 
    def __init__(self):

        self.action_space = spaces.Discrete(2)
        # self.observation_space = spaces.Box(-high, high, dtype=np.float32)

#         self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None
        self.episode = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self):
        episode_df= df.sample(frac=0.5)
        self.episode = episode_df.drop(columns=['Revenue']).values.tolist()
        self.true_labels = episode_df['Revenue'].values.tolist()
        
        try:
            self.state = self.episode.pop()
        except:
            self.state = None
#         self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)
#     .reshape(-1,3)
    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        
#     try and get the true label from list self.rewards
#     if none then done
        done = False
        true_label = self.true_labels.pop()
    
        if true_label is not None:
            if (true_label == 1) & (action == true_label):
                reward = 1
            elif (true_label == 0) & (action == true_label):
                reward = lamb_da
            elif (true_label == 0) & (action != true_label):
                reward = -lamb_da
            else:
                reward = -1
                done = True
        else:
            reward = 0
            done = True
        
        return np.array(self.state), reward, done, {}
#         else:
#             done = True
            
#         if not done:
#             reward = 1.0
#         elif self.steps_beyond_done is None:
#             # Pole just fell!
#             self.steps_beyond_done = 0
#             reward = 1.0
#         else:
#             if self.steps_beyond_done == 0:
#                 logger.warn("You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.")
#             self.steps_beyond_done += 1
#             reward = 0.0

        



    def render(self, mode='human'):
        
        try:
            self.state = self.episode.pop()
        except:
            self.state = None
            
        if self.state is None: return None

        return np.array(self.state).reshape(-1,3)
    
env = CartPoleEnv()

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
qtable = np.zeros((state_size, action_size))

In [ ]:
total_episodes = 15000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005             # Exponential decay rate for exploration prob

In [ ]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)


In [ ]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()